In [23]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objs as go

fonte dos dados: https://info.dengue.mat.br/services/api

In [2]:
df = pd.read_csv('dengue_1-42.csv')

## Dicionário de Dados:

- data_iniSE: First day of the epidemiological week (Sunday);
- SE: epidemiological week;
- casos_est: Estimated number of cases per week using the nowcasting model (note: Values ​​are updated retrospectively every week);
- casos_est_min e casos_est_max: 95% credibility interval of the estimated number of cases;
- casos: Number of reported cases per week (Values ​​are updated retrospectively every week);
- p_rt1: Probability of (Rt> 1). For issuing the orange alert, we use criterion p_rt1 > 0.95 for 3 weeks or more;
- p_inc100k: Estimated incidence rate per 100,000;
- Localidade_id: Sub-municipality division (currently implemented only in Rio de Janeiro);
- nivel: Alert level (1 = green, 2 = yellow, 3 = orange, 4 = red);
- id: Numerical index;
- versao_modelo: Model version (internal use);
- Rt: Point estimate of the reproductive number of cases, see (Know more);
- pop: estimated population (IBGE);
- tempmin: average of the daily minimum temperatures along the week;
- umidmax: average of the daily maximum relative air humidity along the week;
- receptivo: indicates climate receptivity, i.e., conditions for high vectorial capacity. 0 = unfavorable, 1 = favorable, 2 = - - favorable this week and last week, 3 = favorable for at least three weeks (sufficient for completing a transmission cycle);
- transmissao: evidence of sustained transmission: 0 = no evidence, 1 = possible, 2 = likely, 3 = highly likely;
- nivel_inc: Estimated incidence below the pre-epidemic threshold, 1 = above the pre-epidemic threshold, but below the epidemic threshold, 2 = above the epidemic threshold;
- notif_accum_year: accumulated number of cases per year;

In [3]:
df.head()

,data_iniSE,SE,casos_est,casos_est_min,casos_est_max,casos,p_rt1,p_inc100k,Localidade_id,nivel,...,umidmed,umidmin,tempmed,tempmax,casprov,casprov_est,casprov_est_min,casprov_est_max,casconf,notif_accum_year
0,2023-10-01,202340,588.5,371,1077,173,0.967004,8.721342,0,2,...,82.273222,79.890628,23.146667,23.560000,NaN,NaN,NaN,NaN,NaN,210697
1,2023-09-24,202339,502.0,437,610,365,0.341684,7.439445,0,1,...,78.051703,75.177973,25.027778,25.833333,NaN,NaN,NaN,NaN,NaN,210697
2,2023-09-17,202338,530.0,504,568,473,0.929835,7.854394,0,1,...,74.148035,67.119275,24.434783,26.217391,NaN,NaN,NaN,NaN,NaN,210697
3,2023-09-10,202337,563.0,548,586,530,0.997239,8.343442,0,1,...,77.312465,70.757445,19.666667,20.500000,NaN,NaN,NaN,NaN,NaN,210697
4,2023-09-03,202336,447.0,436,461,427,0.059088,6.624367,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210697


### Análise Exploratória de Dados (EDA):
- Distribuição dos casos reportados e estimados de dengue ao longo do tempo.
- Distribuição da taxa de incidência estimada por 100.000 habitantes ao longo do tempo.
- Distribuição dos níveis de alerta ao longo do tempo.

In [4]:
print(df.isnull().sum())

data_iniSE            0
SE                    0
casos_est             0
casos_est_min         0
casos_est_max         0
casos                 0
p_rt1                 0
p_inc100k             0
Localidade_id         0
nivel                 0
id                    0
versao_modelo         0
tweet                 0
Rt                    0
pop                   0
tempmin               0
umidmax              16
receptivo             0
transmissao           0
nivel_inc             0
umidmed              16
umidmin              16
tempmed              16
tempmax              16
casprov             562
casprov_est         562
casprov_est_min     562
casprov_est_max     562
casconf             562
notif_accum_year      0
dtype: int64


In [5]:
df['Localidade_id'].value_counts()

0    562
Name: Localidade_id, dtype: int64

In [6]:
df.describe(include='all')

,data_iniSE,SE,casos_est,casos_est_min,casos_est_max,casos,p_rt1,p_inc100k,Localidade_id,nivel,...,umidmed,umidmin,tempmed,tempmax,casprov,casprov_est,casprov_est_min,casprov_est_max,casconf,notif_accum_year
count,562,562.000000,562.000000,562.000000,562.000000,562.000000,562.000000,562.000000,562.0,562.000000,...,546.000000,546.000000,546.000000,546.000000,0.0,0.0,0.0,0.0,0.0,562.0
unique,562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2023-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,201815.135231,376.056050,375.414591,377.322064,374.905694,0.483042,5.573005,0.0,1.615658,...,77.182597,59.534802,23.471272,26.751538,NaN,NaN,NaN,NaN,NaN,210697.0
std,NaN,310.832697,793.894926,793.807210,794.477959,793.848794,0.408104,11.765215,0.0,0.853829,...,7.224479,14.815921,2.790471,3.788058,NaN,NaN,NaN,NaN,NaN,0.0
min,NaN,201301.000000,5.000000,5.000000,5.000000,5.000000,0.000000,0.074098,0.0,1.000000,...,49.263441,20.857143,15.603896,17.571429,NaN,NaN,NaN,NaN,NaN,210697.0
25%,NaN,201536.250000,53.250000,53.250000,53.250000,53.250000,0.036894,0.789144,0.0,1.000000,...,73.324273,48.584437,21.503432,24.142857,NaN,NaN,NaN,NaN,NaN,210697.0
50%,NaN,201820.500000,104.500000,104.500000,104.500000,104.500000,0.413582,1.548649,0.0,1.000000,...,77.993688,59.289346,23.524026,27.000000,NaN,NaN,NaN,NaN,NaN,210697.0
75%,NaN,202103.750000,276.250000,276.250000,276.250000,271.500000,0.962680,4.093918,0.0,2.000000,...,82.316340,72.764500,25.441687,29.000000,NaN,NaN,NaN,NaN,NaN,210697.0


In [8]:

df['Localidade_id'].value_counts()

0    562
Name: Localidade_id, dtype: int64

In [9]:
df['nivel'].value_counts()

1    325
2    157
3     51
4     29
Name: nivel, dtype: int64

In [10]:
colunas_com_nulos = ['umidmax','umidmed','umidmin','tempmed','tempmax']

for col in colunas_com_nulos:
    df[col] = df[col].interpolate(method='linear',limit_direction='both')

In [11]:
colunas_a_remover = ['casprov','casprov_est','casprov_est_min','casprov_est_max','casconf']
df.drop(columns=colunas_a_remover, inplace=True)

In [12]:
print(df.isnull().sum())

data_iniSE          0
SE                  0
casos_est           0
casos_est_min       0
casos_est_max       0
casos               0
p_rt1               0
p_inc100k           0
Localidade_id       0
nivel               0
id                  0
versao_modelo       0
tweet               0
Rt                  0
pop                 0
tempmin             0
umidmax             0
receptivo           0
transmissao         0
nivel_inc           0
umidmed             0
umidmin             0
tempmed             0
tempmax             0
notif_accum_year    0
dtype: int64


1. Como o número de casos estimados (casos_est) tem evoluído ao longo do tempo?

2. Em quais semanas epidemiológicas (SE) o nivel de alerta foi o mais alto?

3. A temperatura ou a umidade têm alguma correlação com o número de casos de dengue?

4. Como o número reprodutivo Rt tem se comportado ao longo do tempo? Ele permaneceu abaixo de 1, indicando um controle da epidemia?

5. Existem padrões sazonais nos dados?

### 1. Como o número de casos estimados (casos_est) tem evoluído ao longo do tempo?

In [16]:
fig = px.line(df,
              x='data_iniSE',
              y='casos_est',
              title='Evolução do número de casos estimados de dengue ao longo do tempo na cidade do Rio de Janeiro',
              labels={'data_iniSE': 'Data', 'casos_est': 'Número de casos estimados'})

fig.show()

### 2. Em quais semanas epidemiológicas (SE) o nivel de alerta foi o mais alto?

In [27]:
fig = px.line(df, 
              x='data_iniSE', 
              y='nivel', 
              title='Evolução do Nível de Alerta ao Longo das Semanas Epidemiológicas',
              labels={'data_iniSE':'Data','nivel':'Nível de Alerta'})

# Adicionar um marcador para as semanas com nível máximo de alerta
scatter_trace = go.Scatter(x=df[df['nivel'] == 4]['data_iniSE'], y=df[df['nivel'] == 4]['nivel'], mode='markers', marker=dict(color='red'))

fig.add_trace(scatter_trace)

# Ajustar o layout e mostrar o gráfico
fig.update_layout(
    showlegend=False,
    yaxis=dict(
        tickvals=[1, 2, 3, 4],  # Set the specific y-axis tick values
        title='Nível'  # Add a y-axis title
    )
)

fig.show()

### 3. A temperatura ou a umidade têm alguma correlação com o número de casos de dengue?

#### calcular o coeficiente de correlação:

Para determinar se a temperatura ou a umidade têm alguma correlação com o número de casos de dengue, podemos calcular o coeficiente de correlação entre essas variáveis. Vamos usar o coeficiente de correlação de Pearson, que varia de -1 a 1. Um valor próximo de 1 indica uma correlação positiva forte; um valor próximo de -1 indica uma correlação negativa forte; e um valor próximo de 0 indica nenhuma correlação.

In [28]:
corr_temp = df['casos'].corr(df['tempmed'])
corr_umid = df['casos'].corr(df['umidmed'])


Para temperatura:

In [39]:
fig_temp = px.scatter(df,
                      x='tempmed',
                      y='casos',
                      title='Casos de Dengue x Temperatura Média em Graus Celcius',
                      labels={'tempmed':'Temperatura Média em °C','casos':'Casos'})

fig_temp.add_annotation(text=f"Correlação: {corr_temp:.2f}",
                        xanchor='right', x=0.97, yanchor='top', y=0.03,
                        showarrow=False, font=dict(color="black"))

fig_temp.show()


Para umidade:

In [40]:
fig_umid = px.scatter(df,
                      x='umidmed',
                      y='casos',
                      title='Casos de Dengue x Umidade Média',
                      labels={'umidmed':'Umidade Média','casos':'Casos'})

fig_umid.add_annotation(text=f"Correlação: {corr_umid:.2f}",
                        xanchor='right', x=0.97, yanchor='top', y=0.03,
                        showarrow=False, font=dict(color="black"))

fig_umid.show()


Em ambos os casos, o coeficiente de correlação está proximo de zero, indicando que não há correlação com o aumento no número de casos

### 4. Como o número reprodutivo Rt tem se comportado ao longo do tempo? Ele permaneceu abaixo de 1, indicando um controle da epidemia?

In [42]:
# Criando o gráfico de linha para Rt ao longo do tempo
fig_rt = px.line(df,
                 x='data_iniSE',
                 y='Rt',
                 title='Evolução do Número Reprodutivo Rt ao Longo do Tempo',
                 labels={'data_iniSE':'Data','Rt':'Numero Reprodutivo'})

# Adicionando linha horizontal em y=1
fig_rt.add_trace(go.Scatter(x=df['data_iniSE'], y=[1]*len(df),
                            mode='lines',
                            line=dict(color='red', dash='dash'),
                            name='Rt = 1'))

# Ajustar o layout
fig_rt.update_layout(yaxis_title="Rt",
                     annotations=[dict(x=0.05, y=1.02, xref='paper', yref='y',
                                       text='Rt=1', showarrow=False, font=dict(color="red"))])

# Mostrar o gráfico
fig_rt.show()


Em vários momentos, o Rt ultrapassa o valor de 1. Isso sugere que o mosquito está transmitindo o vírus da dengue a um ritmo que pode levar a um aumento no número de casos.

Há também vários períodos onde Rt é menor que 1, sugerindo que o número de novas infecções está diminuindo. Isso pode ser devido a fatores como um clima menos favorável para os mosquitos, intervenções eficazes de saúde pública ou outros fatores que reduzem a transmissão..